# Práctica 2 - SVM

## Librerías y dataframes
Primero, cargamos las librerías necesarias y creamos los dataframes a usar

In [2]:
library(tidyverse)

dataframe_a <- data.frame(X1 = c(0, 4), X2 = c(0, 4), Y = c(1, -1))
dataframe_a

dataframe_b <- data.frame(X1 = c(2, 0, 1), X2 = c(0, 0, 1), Y = c(1, -1, -1))
dataframe_b

dataframe_c <- data.frame(X1 = c(2, 2, -2, -2, 1, 1, -1, -1),
                          X2 = c(2, -2, -2, 2, 1, -1, -1, 1),
                          Y = c(rep(1, 8), rep(-1, 4)))
dataframe_c

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


X1,X2,Y
<dbl>,<dbl>,<dbl>
0,0,1
4,4,-1


X1,X2,Y
<dbl>,<dbl>,<dbl>
2,0,1
0,0,-1
1,1,-1


X1,X2,Y
<dbl>,<dbl>,<dbl>
2,2,1
2,-2,1
-2,-2,1
-2,2,1
2,2,1
2,-2,1
-2,-2,1
-2,2,1
1,1,-1


## Funciones auxiliares
Una vez cargadas las librerías y creados los dataframes, podemos crear algunas funciones auxiliares (como por ejemplo, para la transformación).

In [137]:
euclidean <- function(xp, xn) sqrt(sum((xp - xn)^2))

transformacion <- function(vector) {
  x1 <- vector[1]
  print(x1)
  x2 <- vector[2]
  print(x2)
  if (sqrt(x1^2 + x2^2) > 2) {
    vector[1] <- 4 - x2 + abs(x1 - x2)
    vector[2] <- 4 - x1 + abs(x1 - x2)
  }
}

kernel <- function(x1, x2) {
  len_x1 <- length(x1)
  len_x2 <- length(x2)
  k <- matrix(data = NA, nrow = len_x1, ncol = len_x2)
  for (i in 1:len_x1) {
    xi <- c(x1[i], x2[i])
    for (j in 1:len_x2) {
      xj <- c(x1[j], x2[j])
      k[i, j] <- xi %*% xj
    }
  }
  return(k)
}

support_vectors <- function(x1_positive, x2_positive,
                            x1_negative, x2_negative) {
  len_positive <- length(x1_positive)
  len_negative <- length(x1_negative)
  svp <- NA
  svn <- NA
  min_dist <- Inf
  for (i in 1:len_positive) {
    xp <- c(x1_positive[i], x2_positive[i])
    for (j in 1:len_negative) {
      xn <- c(x1_negative[j], x2_negative[j])
      dist <- euclidean(xp, xn)
      if (dist < min_dist) {
        min_dist <- dist
        svp <- xp
        svn <- xn
      }
    }
  }
  return(list(SVP = data.frame(X1 = svp[1], X2 = svp[2]),
              SVN = data.frame(X1 = svn[1], X2 = svn[2])))
}

## Ejercicio 1. Determinar y mostrar los vectores soporte
Para el dataframe A, los vectores soporte son sus dos puntos, pero podemos determinarlos usando una función auxiliar.

In [139]:
dataframe_a_positive <- as.vector(dataframe_a[dataframe_a$Y == 1, -3])
dataframe_a_negative <- as.vector(dataframe_a[dataframe_a$Y == -1, -3])
support_vectors(dataframe_a_positive[[1]], dataframe_a_positive[[2]],
                dataframe_a_negative[[1]], dataframe_a_negative[[2]])

X1,X2
<dbl>,<dbl>
0,0
X1,X2
<dbl>,<dbl>
4,4


De igual forma, para los dataframes B y C podemos usar la función auxiliar para determinar los vectores soporte.

In [140]:
dataframe_b_positive <- as.vector(dataframe_b[dataframe_b$Y == 1, -3])
dataframe_c_positive <- as.vector(dataframe_c[dataframe_c$Y == 1, -3])
dataframe_b_negative <- as.vector(dataframe_c[dataframe_b$Y == -1, -3])
dataframe_c_negative <- as.vector(dataframe_c[dataframe_c$Y == -1, -3])
support_vectors(dataframe_b_positive[[1]], dataframe_b_positive[[2]],
                dataframe_b_negative[[1]], dataframe_b_negative[[2]])
support_vectors(dataframe_c_positive[[1]], dataframe_c_positive[[2]],
                dataframe_c_negative[[1]], dataframe_c_negative[[2]])

X1,X2
<dbl>,<dbl>
2,0
X1,X2
<dbl>,<dbl>
1,1


X1,X2
<dbl>,<dbl>
2,2
X1,X2
<dbl>,<dbl>
1,1
